<h1>Vikramaditya Reddy Varkala</h1>
<h2>Z1973679</h2>

<b1><u><b>Question 1b: Download & Extract Files  </b></u></b1>

In [4]:
import aiohttp
import asyncio
import os
import shutil
import zipfile
import nest_asyncio

nest_asyncio.apply()

async def download(session,url,directory):
    async with session.get(url) as response:
        zip_file=await response.read()
        with open(os.path.basename(url),'wb') as f:
            f.write(zip_file)
        with zipfile.ZipFile(os.path.basename(url),'r') as ref:
            ref.extractall(directory)
# https://docs.python.org/3/library/functions.html#open

async def main():
    directory='artist-data'
    list1=[]
    for i,j in [(75,79),(80,99),(100,119),(120,199),(200,249),(250,999)]:
        list1.append(f"https://faculty.cs.niu.edu/~dakoop/cs503-2023sp/a7/{i}_million_to_{j}_million.zip")
        
#list1 stores contents of https://faculty.cs.niu.edu/~dakoop/cs503-2023sp/a7/ 

    async with aiohttp.ClientSession() as session:
        tasks=[]
        for url in list1:
            if not os.path.exists(os.path.basename(url)):
                tasks.append(asyncio.create_task(download(session,url,directory)))
            elif not os.path.exists(directory):
                with zipfile.ZipFile(os.path.basename(url),'r') as ref:
                    ref.extractall(directory)

        await asyncio.gather(*tasks)
        
#removes contents of artist data if already exists
if os.path.exists('artist-data'):
    shutil.rmtree('artist-data')
asyncio.run(main())

# References:https://docs.aiohttp.org/en/stable/
# https://realpython.com/python-concurrency/#asyncio-version

#using  nest_asyncio to fix  RuntimeError: This event loop is already running
#https://stackoverflow.com/questions/59740704/correct-use-constraints-of-use-of-nest-asyncio
#https://jupyter-tutorial.readthedocs.io/en/stable/performance/asyncio-example.html


<b1><u><b>Question 2: Find Matching Files  </b></u></b1>

In [5]:
import os

contents=[]
for root,dirs,files in os.walk("artist-data"):
    for file in files:
        if file.endswith(".npy"):
            contents.append(os.path.join(root, file))

print("Number of .npy files found:",len(contents))
for num,i in enumerate(contents):
    print(str(num+1)+". "+i)
    
# dirs to printout subdirectories from root
# References:https://www.tutorialspoint.com/python/os_walk.htm

Number of .npy files found: 32
1. artist-data/75_million_to_79_million/United States/kenny-g.npy
2. artist-data/75_million_to_79_million/United States/alabama.npy
3. artist-data/75_million_to_79_million/United States/luke-bryan.npy
4. artist-data/75_million_to_79_million/United States/imagine-dragons.npy
5. artist-data/75_million_to_79_million/United States/gloria-estefan.npy
6. artist-data/75_million_to_79_million/Other/the-weeknd.npy
7. artist-data/100_million_to_119_million/United States/george-strait.npy
8. artist-data/100_million_to_119_million/United States/janet-jackson.npy
9. artist-data/100_million_to_119_million/United Kingdom/george-michael.npy
10. artist-data/100_million_to_119_million/Other/nicki-minaj.npy
11. artist-data/80_million_to_99_million/United States/lionel-richie.npy
12. artist-data/80_million_to_99_million/United States/post-malone.npy
13. artist-data/80_million_to_99_million/United States/ariana-grande.npy
14. artist-data/80_million_to_99_million/United States

<b1><u><b>Question 3: Use Threads to Process Files </b></u></b1>

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
import concurrent.futures

directory=Path("artist-data")
contents=list(directory.rglob("*/*.npy"))
# https://docs.python.org/3/library/pathlib.html:.rglob

def process(file):
    data=np.load(file)
    df=pd.DataFrame(data=data.T,columns=["DATE","VIEWS"])
#Transposing data(data.T) as the data is structured in horizontal format.    
#https://docs.python.org/3/library/pathlib.html .stem
#Replace _ with <space>
    artist_name=file.stem.replace("-"," ").title()
#new column
    df = df.assign(ARTIST=artist_name)
    return df

# Processing using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    res=list(executor.map(process,contents))
# https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor
    
#Concatenate
dataframe=pd.concat(res)

dataframe["DATE"]=pd.to_datetime(dataframe["DATE"],format="%Y%m%d")
#since date is mentioned in yyyymmdd format in part 3 example george-strait.npy
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

#dict1 has dataframes for 12 months.
#key:value
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.month.html
dict1={month:dataframe[dataframe["DATE"].dt.month==month] for month in range(1,13)}

# using location(.)iloc of first date in column to write the file name
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
for month,dframe in dict1.items():
    filename=f"{dframe['DATE'].iloc[0]:%Y%m01}.csv.gz"
    dframe.to_csv(filename,index=False)


In [7]:
#TestCase1
df = pd.read_csv("20221001.csv.gz")
print(df)

          DATE    VIEWS           ARTIST
0   2022-10-01    39196          Kenny G
1   2022-10-01    23612          Alabama
2   2022-10-01    67167       Luke Bryan
3   2022-10-01   127217  Imagine Dragons
4   2022-10-01    93113   Gloria Estefan
5   2022-10-01   286975       The Weeknd
6   2022-10-01   116814    George Strait
7   2022-10-01   193239    Janet Jackson
8   2022-10-01   292658   George Michael
9   2022-10-01   379675      Nicki Minaj
10  2022-10-01   110192    Lionel Richie
11  2022-10-01   275275      Post Malone
12  2022-10-01   423614    Ariana Grande
13  2022-10-01    34171         Flo Rida
14  2022-10-01   237936           R.E.M.
15  2022-10-01   590429          Madonna
16  2022-10-01   484971          Rihanna
17  2022-10-01  1431262     Taylor Swift
18  2022-10-01   326423  Whitney Houston
19  2022-10-01   354651      Celine Dion
20  2022-10-01   246919       Bruno Mars
21  2022-10-01   334706       Katy Perry
22  2022-10-01   231359        Metallica
23  2022-10-01  

In [8]:
#TestCase2
df = pd.read_csv("20220101.csv.gz")
print(df)

          DATE   VIEWS           ARTIST
0   2022-01-01   48071          Kenny G
1   2022-01-01   22984          Alabama
2   2022-01-01   55667       Luke Bryan
3   2022-01-01  107087  Imagine Dragons
4   2022-01-01   57364   Gloria Estefan
5   2022-01-01  730578       The Weeknd
6   2022-01-01   83411    George Strait
7   2022-01-01  681922    Janet Jackson
8   2022-01-01  323710   George Michael
9   2022-01-01  265998      Nicki Minaj
10  2022-01-01   91305    Lionel Richie
11  2022-01-01  158082      Post Malone
12  2022-01-01  631745    Ariana Grande
13  2022-01-01   34297         Flo Rida
14  2022-01-01  218595           R.E.M.
15  2022-01-01  385166          Madonna
16  2022-01-01  614090          Rihanna
17  2022-01-01  567430     Taylor Swift
18  2022-01-01  358395  Whitney Houston
19  2022-01-01  253011      Celine Dion
20  2022-01-01  212096       Bruno Mars
21  2022-01-01  467378       Katy Perry
22  2022-01-01  194394        Metallica
23  2022-01-01  460402        Lady Gaga
